In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/1dl034-project-vt25/sample_submission.csv
/kaggle/input/1dl034-project-vt25/taxi_zone_lookup.csv
/kaggle/input/1dl034-project-vt25/training_dataset.csv
/kaggle/input/1dl034-project-vt25/evaluation_dataset.csv


In [8]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import r2_score, accuracy_score #, root_mean_squared_log_error
#from pyspark.sql import SparkSession
import pandas as pd
from pandas.api.types import is_numeric_dtype as is_num
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# spark = SparkSession.builder.appName("AIProject").config("spark.driver.memory", "4g").getOrCreate()
# data_frame = spark.read.csv('/kaggle/input/1dl034-project-vt25/training_dataset.csv', header=True, inferSchema=True)
# df = data_frame.limit(50000).toPandas()

In [ ]:
df = pd.read_csv('/kaggle/input/1dl034-project-vt25/training_dataset.csv', nrows=50000)


In [ ]:
df = df.dropna()

df['store_and_fwd_flag'] = df['store_and_fwd_flag'].replace(['N', 'Y'], [1, 2])

df = df.drop('ID', axis=1)
df = df.drop('vendorid', axis=1)
df = df.drop('tpep_pickup_datetime', axis=1)
df = df.drop('tpep_dropoff_datetime', axis=1)

df

In [ ]:
numeric_columns = [
    "passenger_count", "trip_distance", "pulocationid", "dolocationid",
    "ratecodeid", "fare_amount", "extra", "mta_tax", "improvement_surcharge",
    "tip_amount", "tolls_amount", "total_amount", "congestion_surcharge", "airport_fee", "duration"
]

for col in numeric_columns:
    assert is_num(df[col])

In [ ]:
x = df.iloc[:, :-1]
y = df["duration"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
print(x.shape)
print(y.shape)

In [ ]:
lin_mod = LinearRegression()

lin_mod.fit(x_train, y_train)

y_pred_lin_mod = lin_mod.predict(x_test)

print(r2_score(y_test, y_pred_lin_mod))

In [ ]:
n_cv = 2

In [ ]:
# TODO: Fix, did not converge
# maybe increase number of iterations
# 4 min 12 sec
log_mod = LogisticRegression(max_iter=200)

log_mod.fit(x_train, y_train)

y_pred_log_mod = log_mod.predict(x_test)

print(accuracy_score(y_test, y_pred_log_mod))

In [ ]:
ranf_mod = RandomForestRegressor(random_state=42)

# print(np.mean(cross_val_score(ranf_mod, x, y, cv=n_cv)))
ranf_mod.fit(x_train, y_train)
y_pred_ranf = ranf_mod.predict(x_test)
# ranf_mod.score(x_test, y_test)

print(r2_score(y_test, y_pred_ranf))


In [ ]:
ranf_mod = RandomForestRegressor(random_state=42)
ranf_mod.fit(x_train, y_train)

y_pred_ranf = ranf_mod.predict(x_test)
print(r2_score(y_test, y_pred_ranf))


feature_scores = pd.Series(ranf_mod.feature_importances_, index = x_train.columns).sort_values(ascending = False)

import seaborn as sns
sns.barplot(x=feature_scores, y=feature_scores.index, )
plt.title("Importance of each Feature for the Random Forest classifier")
plt.xlabel('Importance')
plt.ylabel('Features')
plt.show()




Based on feature importance, features are removed.

In [ ]:


df = df.drop('store_and_fwd_flag', axis=1)
df = df.drop('airport_fee', axis=1)
df = df.drop('mta_tax', axis=1)
df = df.drop('tolls_amount', axis=1)
df = df.drop('improvement_surcharge', axis=1)
df = df.drop('congestion_surcharge', axis=1)
df = df.drop('ratecodeid', axis=1)
df = df.drop('passenger_count', axis=1)

x = df.iloc[:, :-1]
y = df["duration"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

ranf_mod = RandomForestRegressor(random_state=42)
ranf_mod.fit(x_train, y_train)

y_pred_ranf = ranf_mod.predict(x_test)
print(r2_score(y_test, y_pred_ranf))

feature_scores = pd.Series(ranf_mod.feature_importances_, index = x_train.columns).sort_values(ascending = False)

import seaborn as sns
sns.barplot(x=feature_scores, y=feature_scores.index, )
plt.title("Importance of each Feature for the Random Forest classifier")
plt.xlabel('Importance')
plt.ylabel('Features')
plt.show()



In [ ]:
# plt.scatter(df['duration'], df['trip_distance'])
scatter = sns.scatterplot(x='duration', y='trip_distance', data=df)
#scatter.set_xlim(left= 0, right=7000)
# sns.scatterplot(y='duration', x='trip_distance', data=df)

In [ ]:
max_speed = 50
min_duration = 0
cutoff_duration = 3000


df = df[(df['duration']) > min_duration]
df2 = df[(df['trip_distance'] / (df['duration'] / (60 * 60))) < max_speed] # 50 because 50 mph we set as max
df2 = df2.drop(df2[(df2['duration'] > cutoff_duration) & (df2['trip_distance'] / ((df2['duration'] - cutoff_duration) / (60*60)) < max_speed)].index)

scatter = sns.scatterplot(x='duration', y='trip_distance', data=df2)
#scatter.set_xlim(left= 0, right=7000)

In [ ]:
df2

In [ ]:
x = df2.iloc[:, :-1]
y = df2["duration"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

ranf_mod = RandomForestRegressor(random_state=42)
ranf_mod.fit(x_train, y_train)

y_pred_ranf = ranf_mod.predict(x_test)
print(r2_score(y_test, y_pred_ranf))

feature_scores = pd.Series(ranf_mod.feature_importances_, index = x_train.columns).sort_values(ascending = False)

import seaborn as sns
sns.barplot(x=feature_scores, y=feature_scores.index, )
plt.title("Importance of each Feature for the Random Forest classifier")
plt.xlabel('Importance')
plt.ylabel('Features')
plt.show()



In [ ]:
#df2 = df2.drop('total_amount', axis=1)
#df2 = df2.drop('dolocationid', axis=1)
#df2 = df2.drop('pulocationid', axis=1)
#df2 = df2.drop('extra', axis=1)
#df2 = df2.drop('tip_amount', axis=1)
df2 = df2.drop('payment_type', axis=1)

In [ ]:


x = df2.iloc[:, :-1]
y = df2["duration"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

ranf_mod = RandomForestRegressor(random_state=42)
ranf_mod.fit(x_train, y_train)

y_pred_ranf = ranf_mod.predict(x_test)
print('r2', r2_score(y_test, y_pred_ranf))
print('log error', root_mean_squared_log_error(y_test, y_pred_ranf))

feature_scores = pd.Series(ranf_mod.feature_importances_, index = x_train.columns).sort_values(ascending = False)

import seaborn as sns
sns.barplot(x=feature_scores, y=feature_scores.index, )
plt.title("Importance of each Feature for the Random Forest classifier")
plt.xlabel('Importance')
plt.ylabel('Features')
plt.show()



In [ ]:
svr_mod = SVR()

# print(np.mean(cross_val_score(svr_mod, x, y, cv=n_cv)))

svr_mod.fit(x_train, y_train)
y_pred_svr = svr_mod.predict(x_test)

# print(svr_mod.score(x_test, y_test))

print(r2_score(y_test, y_pred_svr))

#plt.scatter(y_test, y_pred_svr)

In [ ]:
linsvr_mod = LinearSVR(random_state=42)

# print(np.mean(cross_val_score(linsvr_mod, x, y, cv=n_cv)))

linsvr_mod.fit(x_train, y_train)
# y_pred_lin_svr = linsvr_mod.predict(x_test)

# print(r2_score(y_test, y_pred_lin_svr))

print(linsvr_mod.score())

In [ ]:
kneigh_mod = KNeighborsRegressor()

# print(np.mean(cross_val_score(kneigh_mod, x, y, cv=n_cv)))



In [ ]:
tree_mod = DecisionTreeClassifier()

print(np.mean(cross_val_score(tree_mod, x, y, cv=n_cv)))